In [39]:
import yfinance as yf
import pandas as pd
import datetime
import utils

In [36]:
df = pd.read_csv('../datasets/stocks.csv')
stock_names = df['ticker']
df.head()

,company,stock,link,ticker
0,Tesla Inc,Tesla,https://www.investing.com/equities/tesla-motors,TSLA
1,Apple Inc,Apple,https://www.investing.com/equities/apple-compu...,AAPL
2,Beyond Meat Inc,Beyond Meat,https://www.investing.com/equities/beyond-meat...,BYND
3,Coinbase Global Inc,Coinbase Global,https://www.investing.com/equities/coinbase-gl...,COIN
4,Salesforce Inc,Salesforce Inc,https://www.investing.com/equities/salesforce-com,CRM


In [46]:
def get_stock_info(ticker, start_date, end_date):
    # Fetch stock data
    stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    # Extract high, low, close, and other information
    high = stock_data['High']
    low = stock_data['Low']
    close = stock_data['Close']
    volume = stock_data['Volume']
    open = stock_data['Open']    
    return high, low, close, volume, open

In [47]:
start_date = '2023-12-01'
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
for stock_name in stock_names:
    high, low, close, volume, open = get_stock_info(stock_name, start_date, end_date)
    df = pd.DataFrame({'high': high, 'low': low, 'close': close, 'volume': volume, 'open': open})
    utils.create_path(f'../datasets/stock_data/')
    df.to_csv(f'../datasets/stock_data/{stock_name}.csv')

In [48]:
df.head()

,high,low,close,volume,open
Date,,,,,
2023-12-01,554.700012,545.159973,547.159973,3461700,550.419983
2023-12-04,551.869995,543.039978,548.280029,2709200,548.260010
2023-12-05,552.530029,545.349976,550.260010,2645600,549.099976
2023-12-06,551.880005,547.099976,549.530029,2239700,547.559998
2023-12-07,551.510010,544.520020,548.270020,2511700,551.349976
